In [1]:
# https://openrouteservice.org/
# https://openrouteservice.org/dev/#/api-docs/v2/directions/{profile}/get

# username = dconly
# email = dconly@sacog.org

import requests
import os

import geopandas as gpd
import arcpy
arcpy.env.overwriteOutput = True

api_key_source = r"C:\Users\dconly\GitRepos\GIS-tools\ORS\api2_DO_NOT_COMMIT.txt"
with open(api_key_source) as f:
    ors_api_key = f.readline()







In [3]:
# Make isochrone around single point (NOT necessary to run for making line-based isochrone)
# https://openrouteservice.org/dev/#/api-docs/isochrones

travel_mode = "foot-walking"

orgn_lat = 38.59312635026946
orgn_lon = -121.4487934112549

max_time_mins = 15
max_time_sec = max_time_mins * 60


# IMPORTANT NOTE!!! CAN ENTER ARRAY OF MULTIPLE LAT/LONGS, SO COULD INSERT MULTIPLE POINTS TO GET LINE-BASED ISOCHRONE
body = {"locations":[[orgn_lon, orgn_lat]], "range":[max_time_sec], "range_type":"time"}

headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': ors_api_key,
    'Content-Type': 'application/json; charset=utf-8'
}

call = requests.post(f'https://api.openrouteservice.org/v2/isochrones/{travel_mode}', json=body, headers=headers)

polygon_txt = call.text

In [15]:
src = r'I:\Projects\Darren\PEP\PEP_GIS\PEP_GIS.gdb\test_sr51'
dest = os.path.join(arcpy.env.scratchGDB, 'TEST_sr51')

arcpy.management.CopyFeatures(src, dest)

<Result 'C:\\Users\\dconly\\AppData\\Local\\Temp\\scratch.gdb\\TEST_sr51'>

In [2]:
# Make isochrone around multiple points along a line
# https://openrouteservice.org/dev/#/api-docs/isochrones

# input project line feature class
line_fc = r"I:\Projects\Darren\PEP\PEP_GIS\PEP_GIS.gdb\test_sr51"
sref_wgs84 = arcpy.SpatialReference(4326)


# make temporary feature class of points at regular intervales along lines
# FYI, time permitting, the shapely library has some options for doing this that *might* be faster than ESRI tool
temp_pt_fc = os.path.join(arcpy.env.scratchGDB, "TEMP_pts")
arcpy.management.GeneratePointsAlongLines(line_fc, 
                                          temp_pt_fc, "DISTANCE", 
                                          Distance="1000 feet", 
                                          Include_End_Points="END_POINTS")

# calc x/y coords in WGS84 (WKID 4326) for compatibility with ORS API
pt_fl = "pt_fl"
arcpy.MakeFeatureLayer_management(temp_pt_fc, pt_fl)
arcpy.AddGeometryAttributes_management(Input_Features=pt_fl, 
                                       Geometry_Properties=['POINT_X_Y_Z_M'],
                                      Coordinate_System=sref_wgs84)

# print([f.name for f in arcpy.ListFields(temp_pt_fc)])

# make array of points at regular intervals along line to
line_pts = []
with arcpy.da.SearchCursor(pt_fl, ["POINT_X", "POINT_Y"]) as cur:
    for row in cur:
        lon = row[0]
        lat = row[1]
        pt_coords = [lon, lat]
        line_pts.append(pt_coords)
        
# batchify points into groups of 5, because ORS API cannot process more than 5 points in single call

line_pts_batched = [line_pts[i:i+5] for i, v in enumerate(line_pts) if i % 5 == 0]
# line_pts_batched


                    



In [3]:
# generate isochrones around each of those points
max_time_mins = 15
max_time_sec = max_time_mins * 60
travel_mode = "driving-car" # "driving-car" #"foot-walking"

gdf_master = gpd.GeoDataFrame()

# Go through each batch of 5 points and draw an isochrone around them, then combine all the batches together
# into 1 geodatframe with all relevant isochrone polygons in it. Next step would then be dissolve all polygons.
for pts_batch in line_pts_batched:

    body = {"locations":pts_batch, "range":[max_time_sec], "range_type":"time"}

    headers = {
        'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
        'Authorization': ors_api_key,
        'Content-Type': 'application/json; charset=utf-8'
    }

    call = requests.post(f'https://api.openrouteservice.org/v2/isochrones/{travel_mode}', json=body, headers=headers)

    polygon_txt = call.text
    gdf_batch = gpd.read_file(polygon_txt)
    gdf_batch['dissolve_col'] = 0
    gdf_master = gdf_master.append(gdf_batch)
    
# gdf_master.head(14)
# gdf.plot(cmap='Set1')



In [13]:
# Dissolve the multiple polygons into single polygon using Geopandas
# gdf_master.dissolve(by='value')

gdf_master

,group_index,value,geometry,dissolve_col
0,0,900.0,"POLYGON ((-121.71354 38.55061, -121.71274 38.5...",0
1,1,900.0,"POLYGON ((-121.71103 38.55118, -121.71024 38.5...",0
2,2,900.0,"POLYGON ((-121.70856 38.55174, -121.70776 38.5...",0
3,3,900.0,"POLYGON ((-121.70612 38.55230, -121.70532 38.5...",0
4,4,900.0,"POLYGON ((-121.70365 38.55286, -121.70286 38.5...",0
0,0,900.0,"POLYGON ((-121.70119 38.55342, -121.70042 38.5...",0
1,1,900.0,"POLYGON ((-121.69887 38.55388, -121.69837 38.5...",0
2,2,900.0,"POLYGON ((-121.69798 38.55400, -121.69748 38.5...",0
3,3,900.0,"POLYGON ((-121.64448 38.56432, -121.64383 38.5...",0
4,4,900.0,"POLYGON ((-121.65028 38.56327, -121.64965 38.5...",0


In [19]:
# Dissolve the multiple polygons into single polygon using ESRI Spatially Enabled DataFrame
# NOTE that this is only because it's proving, as of 10/29/2021, maddeningly difficult to get
# geopandas's .dissolve() method to work. It keeps giving a "LooseVersion" error.

# UPDATE 10/31/2021 - Geopandas dissolve() finally was able to work on Darren's home macbook after doing following:
# Create new, blank conda env > install geopandas 0.10.0 > that's it!


# https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#arcgis.features.GeoAccessor.from_layer

# load the gpd geodataframe into esri spatially enabled data frame
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor
arcpy.env.overwriteOutput = True

# convert geopandas geodataframe to esri spatially-enabled geodataframe
sedf = GeoAccessor.from_geodataframe(gdf_master, inplace=False, column_name='SHAPE')

temp_fc = os.path.join(arcpy.env.scratchGDB, "TEMP_fc")

# better way to dissolve the spatially-enabled geodataframe?
sedf.spatial.to_featureclass(temp_fc)
arcpy.management.Dissolve(temp_fc, out_diss_poly)


<Result 'C:\\Users\\dconly\\AppData\\Local\\Temp\\scratch.gdb\\TEST_combPoly20211011_1109'>

In [18]:
# merge those isochrones together into single isochrone, which represents the "affected area" of a project
# 10/10/2021 - THIS IS A CLUNKY WAY TO DO THIS. For some reason Geopandas dissolve isn't working, so
# current workflow is gdf > geojson > arcpy featureclass > dissolved arcpy feature class.
# ideally could be gdf > dissolved gdf > geojson
import datetime as dt
time_sufx = str(dt.datetime.now().strftime('%Y%m%d_%H%M'))
out_diss_poly = os.path.join(arcpy.env.scratchGDB, f"TEST_combPoly{time_sufx}")

arcpy.env.overwriteOutput = True
temp_gjson = "polys.json"# os.path.join(arcpy.env.scratchFolder, "polys.json")
temp_polys_fc = os.path.join(arcpy.env.scratchGDB, "TEMP_polys")

json_temp = gdf_master.to_file(temp_gjson, driver="GeoJSON")

arcpy.conversion.JSONToFeatures(temp_gjson, temp_polys_fc)
arcpy.management.Dissolve(temp_polys_fc, out_diss_poly)

<Result 'C:\\Users\\dconly\\AppData\\Local\\Temp\\scratch.gdb\\TEST_combPoly20211011_1109'>

In [1]:
# UPDATE 10/31/2021 - Geopandas dissolve() finally was able to work on Darren's home macbook after doing following:
# Create new, blank conda env > install geopandas 0.10.0 > that's it!

import geopandas as gpd

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

world = world[['continent', 'geometry']]
world.head()

# continents = world.dissolve(by='continent')

# continents.head()

,continent,geometry
0,Oceania,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000..."
1,Africa,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."
2,Africa,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
3,North America,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742..."
4,North America,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000..."


In [2]:
continents = world.dissolve(by='continent')

In [3]:
continents.head()


,geometry
continent,
Africa,"MULTIPOLYGON (((-11.43878 6.78592, -11.70819 6..."
Antarctica,"MULTIPOLYGON (((-61.13898 -79.98137, -60.61012..."
Asia,"MULTIPOLYGON (((48.67923 14.00320, 48.23895 13..."
Europe,"MULTIPOLYGON (((-53.55484 2.33490, -53.77852 2..."
North America,"MULTIPOLYGON (((-155.22217 19.23972, -155.5421..."
